In [ ]:
# -*- coding: utf-8 -*-
"""
스팀 리뷰 EDA (중앙값/왜도 + 가설 검증용 요약)
- 입력: 같은 폴더의 'weighted_score_above_08.csv.zip'
- 출력: ./eda_out/ 에 요약표 및 그래프 PNG/CSV 저장
"""

import os
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------
# 0) 기본 설정 (한글 폰트/경로)
# ---------------------------
OUTDIR = "eda_out"
os.makedirs(OUTDIR, exist_ok=True)

# 한글 폰트 설정(운영체제별 후보 자동 선택)
import matplotlib
from matplotlib import font_manager, rcParams

def _set_korean_font():
    candidates = [
        "Malgun Gothic",       # Windows
        "AppleGothic",         # macOS
        "NanumGothic",         # Linux common
        "Noto Sans CJK KR",    # Linux alternative
    ]
    found = None
    for name in candidates:
        try:
            fm = font_manager.FontProperties(family=name)
            rcParams["font.family"] = fm.get_name()
            found = fm.get_name()
            break
        except Exception:
            pass
    # 마이너스 부호 깨짐 방지
    rcParams["axes.unicode_minus"] = False
    return found

_set_korean_font()

# ---------------------------
# 1) 데이터 로드
# ---------------------------
FNAME = "weighted_score_above_08.csv"  # 같은 폴더에 있다고 가정
compression = "zip" if FNAME.endswith(".zip") else None
df = pd.read_csv(FNAME, compression=compression, low_memory=False)

# ---------------------------
# 2) 타입 정리 & 파생 변수
# ---------------------------
# 숫자형으로 쓰는 컬럼들
num_cols = [
    "author_playtime_at_review",
    "author_playtime_forever",
    "author_num_reviews",
    "author_num_games_owned",
    "votes_up",
    "votes_funny",
    "voted_up"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 리뷰 길이
if "review" in df.columns:
    df["review_length"] = df["review"].astype(str).str.len()

# 안전: 결측치가 너무 많으면 qcut 에러 방지용 0 대체 (분위수 계산용)
if "author_playtime_at_review" in df.columns:
    df["_playtime_at_rev_filled"] = df["author_playtime_at_review"].fillna(0)

# ---------------------------
# 3) 왜도 함수 (Bowley: 사분위 기반, 이상치에 강건)
# ---------------------------
def bowley_skew(x: np.ndarray) -> float:
    x = np.asarray(x)
    x = x[~np.isnan(x)]
    if x.size == 0:
        return np.nan
    q1, q2, q3 = np.percentile(x, [25, 50, 75])
    iqr = q3 - q1
    return 0.0 if iqr == 0 else (q3 + q1 - 2.0*q2) / iqr

# ---------------------------
# 4) 전체 요약표 (중앙값/왜도)
# ---------------------------
summary_targets = [
    "author_playtime_at_review",
    "author_playtime_forever",
    "author_num_reviews",
    "author_num_games_owned",
    "votes_up",
    "votes_funny",
    "review_length",
]

summary = {}
for col in summary_targets:
    if col not in df.columns: 
        continue
    x = pd.to_numeric(df[col], errors="coerce").dropna().values
    if x.size == 0:
        continue
    q1, med, q3 = np.percentile(x, [25, 50, 75])
    summary[col] = {
        "count": int(x.size),
        "mean": float(np.mean(x)),
        "median": float(med),
        "Q1": float(q1),
        "Q3": float(q3),
        "P95": float(np.percentile(x, 95)),
        "Bowley_skew": float(bowley_skew(x))
    }

summary_df = pd.DataFrame(summary).T
summary_csv = os.path.join(OUTDIR, "summary_median_skew.csv")
summary_df.to_csv(summary_csv, encoding="utf-8-sig")
print(f"[저장] 중앙값/왜도 요약 → {summary_csv}")

# ---------------------------
# 5) 가설별 EDA
# ---------------------------
def save_bar(x, y, title, xlabel, ylabel, fname):
    plt.figure(figsize=(7, 4.5))
    plt.bar(x, y)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.ylim(0, 1 if ylabel.startswith("긍정률") else None)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTDIR, fname), dpi=150)
    plt.close()

def save_scatter(x, y, title, xlabel, ylabel, fname, logx=False, logy=False):
    plt.figure(figsize=(6.5, 5))
    plt.scatter(x, y, s=8, alpha=0.5)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if logx:
        plt.xscale("log")
    if logy:
        plt.yscale("log")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTDIR, fname), dpi=150)
    plt.close()

# ---- [가설1] 플레이타임↑ → 긍정률↑ (개별 리뷰 기준 분위수 비교)
if {"_playtime_at_rev_filled", "voted_up"}.issubset(df.columns):
    # 4분위
    df["playtime_bin"] = pd.qcut(df["_playtime_at_rev_filled"], 4, labels=["Q1","Q2","Q3","Q4"], duplicates="drop")
    g1 = df.groupby("playtime_bin")["voted_up"].mean().reset_index()
    g1_csv = os.path.join(OUTDIR, "H1_playtime_quartiles_positive_rate.csv")
    g1.to_csv(g1_csv, index=False, encoding="utf-8-sig")
    print(f"[저장] 가설1 결과표 → {g1_csv}")
    save_bar(
        x=g1["playtime_bin"],
        y=g1["voted_up"],
        title="가설1: 플레이타임 분위수별 긍정률",
        xlabel="플레이타임 분위수 (리뷰 시점)",
        ylabel="긍정률 (voted_up 비율)",
        fname="H1_playtime_quartiles_positive_rate.png"
    )

# ---- 공통: appid 존재 여부
has_appid = "appid" in df.columns

# ---- [가설2] 게임별 '플레이타임 중앙값'↑ → 긍정률↑
if has_appid and {"author_playtime_at_review","voted_up"}.issubset(df.columns):
    g2 = df.groupby("appid").agg(
        playtime_median=("author_playtime_at_review", "median"),
        positive_rate=("voted_up", "mean"),
        n=("voted_up", "size")
    ).reset_index()
    g2 = g2.dropna(subset=["playtime_median"])
    # 스피어만 상관
    if g2.shape[0] > 2:
        rho = g2[["playtime_median", "positive_rate"]].corr(method="spearman").iloc[0,1]
    else:
        rho = np.nan
    g2["spearman_rho_all_pairs"] = rho
    g2_csv = os.path.join(OUTDIR, "H2_game_playtime_median_vs_positive_rate.csv")
    g2.to_csv(g2_csv, index=False, encoding="utf-8-sig")
    print(f"[저장] 가설2 결과표 (Spearman rho={rho:.3f}) → {g2_csv}")
    save_scatter(
        x=g2["playtime_median"].clip(lower=1), 
        y=g2["positive_rate"],
        title=f"가설2: 게임별 플레이타임 중앙값 vs 긍정률 (Spearman={rho:.3f})",
        xlabel="게임별 플레이타임 중앙값(분)",
        ylabel="긍정률",
        fname="H2_playtime_median_vs_positive_rate.png",
        logx=True, logy=False
    )

# ---- [가설3] 게임별 '플레이타임 왜도'↑ → 긍정률↓
if has_appid and {"author_playtime_at_review","voted_up"}.issubset(df.columns):
    # 그룹별 Bowley 왜도 계산
    def _bowley_group(s: pd.Series) -> float:
        return bowley_skew(pd.to_numeric(s, errors="coerce").values)
    g3_skew = df.groupby("appid")["author_playtime_at_review"].apply(_bowley_group).reset_index(name="playtime_bowley_skew")
    g3_pos = df.groupby("appid")["voted_up"].mean().reset_index(name="positive_rate")
    g3 = pd.merge(g3_skew, g3_pos, on="appid", how="inner")
    g3 = g3.replace([np.inf, -np.inf], np.nan).dropna()
    rho3 = g3[["playtime_bowley_skew","positive_rate"]].corr(method="spearman").iloc[0,1] if g3.shape[0] > 2 else np.nan
    g3["spearman_rho_all_pairs"] = rho3
    g3_csv = os.path.join(OUTDIR, "H3_game_playtime_skew_vs_positive_rate.csv")
    g3.to_csv(g3_csv, index=False, encoding="utf-8-sig")
    print(f"[저장] 가설3 결과표 (Spearman rho={rho3:.3f}) → {g3_csv}")
    save_scatter(
        x=g3["playtime_bowley_skew"],
        y=g3["positive_rate"],
        title=f"가설3: 게임별 플레이타임 왜도 vs 긍정률 (Spearman={rho3:.3f})",
        xlabel="게임별 플레이타임 왜도 (Bowley)",
        ylabel="긍정률",
        fname="H3_playtime_skew_vs_positive_rate.png",
        logx=False, logy=False
    )

# ---- [가설4] 리뷰 수/보유량 ↑ → 더 비판적 (긍정률↓) : 개별 리뷰 기반 분위수 비교
def _bin_positive_rate(var_name: str, label: str):
    if var_name not in df.columns: 
        return
    temp = df[[var_name, "voted_up"]].dropna()
    if temp.empty:
        return
    # 분위수 구간
    try:
        bins = pd.qcut(temp[var_name], 4, labels=["Q1","Q2","Q3","Q4"], duplicates="drop")
    except ValueError:
        # unique 값이 적으면 3분위 시도
        bins = pd.qcut(temp[var_name], 3, labels=["Q1","Q2","Q3"], duplicates="drop")
    res = temp.groupby(bins)["voted_up"].mean().reset_index()
    csv_path = os.path.join(OUTDIR, f"H4_{var_name}_quartiles_positive_rate.csv")
    res.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"[저장] 가설4({label}) 결과표 → {csv_path}")
    save_bar(
        x=res[var_name], y=res["voted_up"],
        title=f"가설4: {label} 분위수별 긍정률",
        xlabel=f"{label} 분위수",
        ylabel="긍정률 (voted_up 비율)",
        fname=f"H4_{var_name}_quartiles_positive_rate.png"
    )

if "author_num_reviews" in df.columns and "voted_up" in df.columns:
    _bin_positive_rate("author_num_reviews", "리뷰 수")
if "author_num_games_owned" in df.columns and "voted_up" in df.columns:
    _bin_positive_rate("author_num_games_owned", "보유 게임 수")

# ---- [가설5] votes_funny 왜도↑ → 번역 리뷰의 부정 키워드↑ (텍스트 완료되면 활용)
# 여기서는 베이스라인: 게임별 votes_funny 왜도만 먼저 계산해서 저장
if has_appid and "votes_funny" in df.columns:
    g5 = df.groupby("appid")["votes_funny"].apply(lambda s: bowley_skew(pd.to_numeric(s, errors="coerce").values)).reset_index(name="funny_bowley_skew")
    g5_csv = os.path.join(OUTDIR, "H5_game_votes_funny_skew.csv")
    g5.to_csv(g5_csv, index=False, encoding="utf-8-sig")
    print(f"[저장] 가설5(사전준비) 결과표 → {g5_csv}")
    # 번역 완료 후, 키워드 빈도와 조인해 상/하위 분위 비교하면 됨.

print("\n✅ 완료: 결과 파일은 eda_out/ 폴더를 확인하세요.")
